This doc

*   shows what is AI Agent ?
*   shows why is Agent important ?
*   propose using multiple agents as a crew
*   build a crew (multiple agents system) for research and writing



### 1. What is Agent ?

When people asks questions (prompts) to ChatGPT, it is the people that asks question. It is the ChatGPT that answers the question.

But **Agent** is different.

**Agent asks questions and answers the questions by themself.**

In addition, **Agent can interact with external world. Agent can use tools**. This allows agent to do more.

### 2. Why is Agent important ?

It is the 3rd stage for the development of AGI.

1. Chatbots, AI with conversational language.  
2. Reasoners, human-level problem solving.  
3. **Agents, systems that can take actions**.   
4. Innovators, AI that can aid in invention.  
5. Organizations: AI that can do the work of an organization.  

### 3. Why not use multiple Agents ?

Why not create multiple agents ? No harm.

Let each agent do a specific subtask.

Let these agents work together to achieve the task.

### 4. Let's create multiple Agents to research and write an article!!

In [ ]:
!pip install -q crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

from crewai import Agent, Task, Crew

In [ ]:
from langchain_community.llms import HuggingFaceHub

HF_TOKEN_HERE = '<YOUR_HF_TOKEN_HERE>'

huggingface = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token=HF_TOKEN_HERE,
    task="text-generation",
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


In [ ]:
## Agent: Planner

planner = Agent(
    role = 'Content Planner', # LLM performs better in role playing
    goal = 'Plan engaging and factually accurate content on {topic}',
    backstory = "You're working on planning a blog article "
                "about the topic: {topic}. "
                "You collect information that helps the "
                "audience learn something "
                "and make informed decisions. "
                "Your work is the basis for "
                "the Content Writer to write an article on this topic.",
    allow_delegation=False,
    verbose=True,
    llm=huggingface,
    max_rpm=1,
)

In [ ]:
## Agent: Writter

writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm=huggingface,
    max_rpm=1,
)

In [ ]:
# Agent: Editor

editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm=huggingface,
    max_rpm=1,
    max_retry_limit=3,
)

In [ ]:
# Create Task for Planner Agent

plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
    max_rpm=1,
)

In [ ]:
# Create Task for Writer Agent

write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		    "3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
    max_rpm=1,
)

In [ ]:
# Create Task for Editor Agent
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [ ]:
# Create a crew
crew = Crew(
    agents=[planner, writer],
    tasks=[plan, write],
)

In [ ]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})



> Entering new CrewAgentExecutor chain...
 [INFO]: Max RPM reached, waiting for next minute to start.
You are Content Planner. You're working on planning a blog article about the topic: Artificial Intelligence. You collect information that helps the audience learn something and make informed decisions. Your work is the basis for the Content Writer to write an article on this topic.
Your personal goal is: Plan engaging and factually accurate content on Artificial IntelligenceTo give my best complete final answer to the task use the exact following format:

Thought: I now can give a great answer
Final Answer: my best complete final answer to the task.
Your final answer must be the great and the most complete as possible, it must be outcome described.

I MUST use these formats, my job depends on it!
Current Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. D

In [ ]:
from IPython.display import Markdown
Markdown(result)

Artificial Intelligence (AI) is revolutionizing various industries by enabling more accurate diagnoses, personalized treatments, and faster drug discovery in healthcare, more accurate risk assessments, fraud detection, and personalized financial advice in finance, and more personalized learning experiences, adaptive assessments, and intelligent tutoring systems in education. According to reports by Accenture, PwC, and MarketsandMarkets, AI is expected to save the healthcare industry $150 billion annually by 2026, generate $140 billion in additional value annually by 2025 in finance, and grow from $1.1 billion in 2020 to $4.2 billion by 2025 in education.

Google, Amazon, and Microsoft are leading players in the AI industry, with their AI subsidiaries, DeepMind, AWS, and Azure, respectively, making significant advancements in healthcare, finance, and climate change. Noteworthy news in the AI industry includes Google's DeepMind achieving human-level performance in protein folding, Amazon's Alexa learning to understand sarcasm, and Microsoft's AI system predicting COVID-19 patients' risk of death.

As AI continues to transform various industries, it is essential for professionals to adapt and thrive in this new era of work. Professionals should learn about AI, collaborate with AI, and adapt to AI by learning new skills and adopting new roles. By doing so, they can remain relevant and valuable in their organizations and leverage the strengths of AI and their own human intelligence to achieve better outcomes.

In conclusion, AI is transforming various industries, and professionals should embrace this new era of work and leverage the strengths of AI and their own human intelligence to achieve better outcomes. As AI continues to transform various industries, it is essential for professionals to adapt and thrive in this new era of work.